# Механизм внимания

## Задание

Решить задачу перевода с помощью механизма внимания

In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import spacy
from collections import Counter
from nltk.translate.bleu_score import corpus_bleu

1. **Загрузка и предварительная обработка данных**

In [2]:
def load_data(file_path):
    eng_sentences = []
    rus_sentences = []
    # Открываем файл с датасетом
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Разбиваем строку по табуляции
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                # Добавляем предложения в соответствующие списки, приводим к нижнему регистру
                eng_sentences.append(parts[0].lower())
                rus_sentences.append(parts[1].lower())
    return eng_sentences, rus_sentences

# Путь к датасету 
data_path = 'C:/Users/Yaros/Downloads/rus-eng/rus.txt'

# Загружаем данные
eng_sentences, rus_sentences = load_data(data_path)

# Проверяем количество загруженных предложений
print(f"Количество английских предложений: {len(eng_sentences)}")
print(f"Количество русских предложений: {len(rus_sentences)}")

Количество английских предложений: 496059
Количество русских предложений: 496059


##### Проверяем данные

In [3]:
import random

index = random.randint(0, len(eng_sentences) - 1)
print(f"Английское предложение: {eng_sentences[index]}")
print(f"Русское предложение: {rus_sentences[index]}")

Английское предложение: in my opinion, we've already done more than enough.
Русское предложение: по-моему, мы уже сделали более чем достаточно.


In [4]:
empty_eng = [i for i, sent in enumerate(eng_sentences) if not sent.strip()]
empty_rus = [i for i, sent in enumerate(rus_sentences) if not sent.strip()]

print(f"Индексы пустых английских предложений: {empty_eng}")
print(f"Индексы пустых русских предложений: {empty_rus}")

Индексы пустых английских предложений: []
Индексы пустых русских предложений: []


##### Токенизация с использованием spaCy

In [5]:
# Загружаем модели spaCy для английского и русского языков
spacy_en = spacy.load('en_core_web_sm')
spacy_ru = spacy.load('ru_core_news_sm')

# Токенизаторы на основе spaCy
def tokenize_eng(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_rus(text):
    return [tok.text for tok in spacy_ru.tokenizer(text)]

##### Построение словарей

In [6]:
def build_vocab(tokenized_sentences, min_freq):
    counter = Counter()
    for tokens in tokenized_sentences:
        counter.update(tokens)
    # Начальные специальные токены
    vocab = {'<pad>': 0, '<bos>': 1, '<eos>': 2, '<unk>': 3}
    index = 4
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = index
            index += 1
    return vocab

# Токенизируем все предложения
eng_tokenized = [tokenize_eng(sentence) for sentence in eng_sentences]
rus_tokenized = [tokenize_rus(sentence) for sentence in rus_sentences]

# Строим словари с минимальной частотой вхождения
eng_vocab = build_vocab(eng_tokenized, min_freq=2)
rus_vocab = build_vocab(rus_tokenized, min_freq=2)

# Выводим размеры словарей
print(f"Размер английского словаря: {len(eng_vocab)}")
print(f"Размер русского словаря: {len(rus_vocab)}")

Размер английского словаря: 12903
Размер русского словаря: 37064


##### Функции для преобразования токенов и индексов

In [7]:
def tokens_to_indexes(tokens, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

def indexes_to_tokens(indexes, vocab):
    inv_vocab = {v: k for k, v in vocab.items()}
    return [inv_vocab.get(index, '<unk>') for index in indexes]

In [8]:
# Кастомный класс Dataset
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, trg_sentences, src_tokenizer, trg_tokenizer, src_vocab, trg_vocab):
        self.src_sentences = src_sentences
        self.trg_sentences = trg_sentences
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, idx):
        src_sentence = self.src_sentences[idx]
        trg_sentence = self.trg_sentences[idx]
        
        # Токенизируем и добавляем специальные токены
        src_tokens = ['<bos>'] + self.src_tokenizer(src_sentence) + ['<eos>']
        trg_tokens = ['<bos>'] + self.trg_tokenizer(trg_sentence) + ['<eos>']
        
        # Преобразуем токены в индексы
        src_indexes = tokens_to_indexes(src_tokens, self.src_vocab)
        trg_indexes = tokens_to_indexes(trg_tokens, self.trg_vocab)
        
        return torch.tensor(src_indexes), torch.tensor(trg_indexes)

# Разделяем данные на обучающую и валидационную выборки
train_eng, val_eng, train_rus, val_rus = train_test_split(
    eng_sentences, rus_sentences, test_size=0.1, random_state=42
)

# Создаем экземпляры датасетов
train_dataset = TranslationDataset(train_eng, train_rus, tokenize_eng, tokenize_rus, eng_vocab, rus_vocab)
val_dataset = TranslationDataset(val_eng, val_rus, tokenize_eng, tokenize_rus, eng_vocab, rus_vocab)

##### Создание DataLoader

In [9]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=eng_vocab['<pad>'])
    trg_batch = pad_sequence(trg_batch, padding_value=rus_vocab['<pad>'])
    return src_batch, trg_batch

# Размер батча
BATCH_SIZE = 32

# Создаем DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

2. **Определение моделей**

##### Модель энкодера

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim  # Сохраняем enc_hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        self.dropout = nn.Dropout(0.5)

    def forward(self, src):
        # src: [src_len, batch_size]
        embedded = self.dropout(self.embedding(src))  # [src_len, batch_size, emb_dim]
        outputs, hidden = self.rnn(embedded)
        # Объединяем последние прямое и обратное скрытые состояния
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)  # [batch_size, enc_hid_dim * 2]
        return outputs, hidden  # outputs для внимания, hidden для инициализации декодера

##### Механизм внимания на основе скалярного произведения

In [11]:
class DotProductAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        # Линейные слои для преобразования размерностей
        self.attn = nn.Linear(enc_hid_dim * 2 + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: [batch_size, dec_hid_dim]
        # encoder_outputs: [src_len, batch_size, enc_hid_dim * 2]
        src_len = encoder_outputs.shape[0]

        # Повторяем скрытое состояние декодера src_len раз
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # [batch_size, src_len, dec_hid_dim]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # [batch_size, src_len, enc_hid_dim * 2]

        # Вычисляем энергию
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, src_len, dec_hid_dim]
        attention = self.v(energy).squeeze(2)  # [batch_size, src_len]

        return torch.softmax(attention, dim=1)  # [batch_size, src_len]

##### Механизм внимания на основе MLP

In [12]:
class MLPAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        # Линейные слои для преобразования размерностей
        self.attn = nn.Linear(enc_hid_dim * 2 + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # Аналогично DotProductAttention
        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)

        return torch.softmax(attention, dim=1)

##### Модель декодера

In [13]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, attention):
        super().__init__()
        self.dec_hid_dim = dec_hid_dim  # Сохраняем dec_hid_dim
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim + enc_hid_dim * 2, dec_hid_dim)

        self.fc_out = nn.Linear(emb_dim + dec_hid_dim + enc_hid_dim * 2, output_dim)

        self.dropout = nn.Dropout(0.5)

    def forward(self, input, hidden, encoder_outputs):
        # input: [batch_size]
        # hidden: [batch_size, dec_hid_dim]
        # encoder_outputs: [src_len, batch_size, enc_hid_dim * 2]

        input = input.unsqueeze(0)  # [1, batch_size]

        embedded = self.dropout(self.embedding(input))  # [1, batch_size, emb_dim]

        # Вычисляем веса внимания
        a = self.attention(hidden, encoder_outputs)  # [batch_size, src_len]
        a = a.unsqueeze(1)  # [batch_size, 1, src_len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # [batch_size, src_len, enc_hid_dim * 2]

        # Вычисляем контекстный вектор
        weighted = torch.bmm(a, encoder_outputs)  # [batch_size, 1, enc_hid_dim * 2]
        weighted = weighted.permute(1, 0, 2)  # [1, batch_size, enc_hid_dim * 2]

        # Соединяем эмбеддинг и контекст
        rnn_input = torch.cat((embedded, weighted), dim=2)  # [1, batch_size, emb_dim + enc_hid_dim * 2]

        # Передаем через GRU
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))  # output: [1, batch_size, dec_hid_dim]

        # Упрощаем размерности
        embedded = embedded.squeeze(0)  # [batch_size, emb_dim]
        output = output.squeeze(0)      # [batch_size, dec_hid_dim]
        weighted = weighted.squeeze(0)  # [batch_size, enc_hid_dim * 2]

        # Предсказываем следующее слово
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))  # [batch_size, output_dim]

        return prediction, hidden.squeeze(0)

##### Модель Seq2Seq

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # Новый линейный слой для преобразования скрытого состояния
        self.fc_hidden = nn.Linear(encoder.enc_hid_dim * 2, decoder.dec_hid_dim)

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [src_len, batch_size]
        # trg: [trg_len, batch_size]

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # Тензор для хранения выходов декодера
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # Передаем через энкодер
        encoder_outputs, hidden = self.encoder(src)

        # Преобразуем скрытое состояние энкодера для декодера
        dec_hidden = torch.tanh(self.fc_hidden(hidden))  # [batch_size, dec_hid_dim]

        # Первый вход декодера - токен <bos>
        input = trg[0, :]

        for t in range(1, trg_len):
            # Передаем через декодер
            output, dec_hidden = self.decoder(input, dec_hidden, encoder_outputs)
            outputs[t] = output
            # Решаем, использовать ли teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

3. **Обучение модели**

##### Инициализация модели и оптимизатора

In [15]:
# Определяем размеры
INPUT_DIM = len(eng_vocab)
OUTPUT_DIM = len(rus_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512

# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Инициализируем механизм внимания
attn = DotProductAttention(ENC_HID_DIM, DEC_HID_DIM)
# Для MLP Attention, замените на:
# attn = MLPAttention(ENC_HID_DIM, DEC_HID_DIM)

# Инициализируем энкодер и декодер
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, attn)

# Создаем модель Seq2Seq с обновленным классом
model = Seq2Seq(enc, dec, device).to(device)

# Инициализируем оптимизатор
optimizer = optim.Adam(model.parameters())

# Функция потерь, игнорируем индекс паддинга
criterion = nn.CrossEntropyLoss(ignore_index=rus_vocab['<pad>'])

##### Функции обучения и оценки

In [16]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, (src, trg) in enumerate(iterator):
        src = src.to(device)
        trg = trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        # output: [trg_len, batch_size, output_dim]
        # trg: [trg_len, batch_size]

        output_dim = output.shape[-1]

        # Убираем первый токен и перестраиваем для вычисления потерь
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].reshape(-1)

        loss = criterion(output, trg)

        loss.backward()

        # Градиентный клиппинг
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, trg) in enumerate(iterator):
            src = src.to(device)
            trg = trg.to(device)

            output = model(src, trg, 0)  # Отключаем teacher forcing

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].reshape(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [17]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("Number of GPUs:", torch.cuda.device_count())
    print("Current GPU:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available.")

PyTorch version: 2.4.0
CUDA available: True
CUDA version: 12.4
Number of GPUs: 1
Current GPU: 0
GPU Name: NVIDIA GeForce RTX 3070 Laptop GPU


##### Цикл обучения

In [18]:
N_EPOCHS = 10
CLIP = 1

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    val_loss = evaluate(model, val_loader, criterion)

    print(f'Эпоха: {epoch+1:02}')
    print(f'\tПотеря на обучении: {train_loss:.3f}')
    print(f'\tПотеря на валидации: {val_loss:.3f}')

Эпоха: 01
	Потеря на обучении: 2.749
	Потеря на валидации: 2.815
Эпоха: 02
	Потеря на обучении: 2.297
	Потеря на валидации: 2.835
Эпоха: 03
	Потеря на обучении: 2.246
	Потеря на валидации: 2.948
Эпоха: 04
	Потеря на обучении: 2.250
	Потеря на валидации: 2.982
Эпоха: 05
	Потеря на обучении: 2.277
	Потеря на валидации: 3.039
Эпоха: 06
	Потеря на обучении: 2.321
	Потеря на валидации: 3.127
Эпоха: 07
	Потеря на обучении: 2.388
	Потеря на валидации: 3.219
Эпоха: 08
	Потеря на обучении: 2.468
	Потеря на валидации: 3.332
Эпоха: 09
	Потеря на обучении: 2.569
	Потеря на валидации: 3.424
Эпоха: 10
	Потеря на обучении: 2.668
	Потеря на валидации: 3.455


4. Оценка качества модели

In [21]:
def translate_sentence(model, sentence, src_tokenizer, src_vocab, trg_vocab, max_len=50):
    model.eval()

    tokens = ['<bos>'] + src_tokenizer(sentence.lower()) + ['<eos>']
    src_indexes = tokens_to_indexes(tokens, src_vocab)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)
        # Преобразуем скрытое состояние энкодера для декодера
        dec_hidden = torch.tanh(model.fc_hidden(hidden))

    trg_indexes = [trg_vocab['<bos>']]

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, dec_hidden = model.decoder(trg_tensor, dec_hidden, encoder_outputs)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == trg_vocab['<eos>']:
            break

    trg_tokens = indexes_to_tokens(trg_indexes, trg_vocab)

    return trg_tokens[1:]

In [22]:
sentence = "Hello, how are you?"
translated_tokens = translate_sentence(model, sentence, tokenize_eng, eng_vocab, rus_vocab)
translated_sentence = ' '.join(translated_tokens)
print(f"Перевод: {translated_sentence}")

Перевод: привет привет , как поживаете ? <eos>


##### Вычисление BLEU Score

In [23]:
def calculate_bleu(model, dataset, src_tokenizer, trg_tokenizer, src_vocab, trg_vocab, n_examples=100):
    trgs = []
    pred_trgs = []

    for i in range(n_examples):
        src_sentence = dataset.src_sentences[i]
        trg_sentence = dataset.trg_sentences[i]
        
        pred_trg = translate_sentence(model, src_sentence, src_tokenizer, src_vocab, trg_vocab)
        pred_trg = [token for token in pred_trg if token not in ['<eos>', '<pad>']]
        
        trg_tokens = trg_tokenizer(trg_sentence)
        
        trgs.append([trg_tokens])
        pred_trgs.append(pred_trg)

    bleu = corpus_bleu(trgs, pred_trgs)
    return bleu

# Вычисляем BLEU Score на валидационном наборе
bleu_score = calculate_bleu(model, val_dataset, tokenize_eng, tokenize_rus, eng_vocab, rus_vocab)
print(f'BLEU Score = {bleu_score*100:.2f}')

BLEU Score = 19.87


In [24]:
bleu_score = calculate_bleu(model, val_dataset, tokenize_eng, tokenize_rus, eng_vocab, rus_vocab, n_examples=100)
print(f'BLEU Score on validation set: {bleu_score*100:.2f}')

BLEU Score on validation set: 19.87


In [25]:
for i in range(5):
    src_sentence = val_dataset.src_sentences[i]
    trg_sentence = val_dataset.trg_sentences[i]
    
    pred_trg = translate_sentence(model, src_sentence, tokenize_eng, eng_vocab, rus_vocab)
    pred_sentence = ' '.join(pred_trg)
    
    print(f"Source: {src_sentence}")
    print(f"Target: {trg_sentence}")
    print(f"Predicted: {pred_sentence}")
    print("-" * 50)

Source: it seems you were right.
Target: похоже, ты была права.
Predicted: кажется , ты правы права . <eos>
--------------------------------------------------
Source: if there are no dogs in heaven, then when i die i want to go where they went.
Target: если в раю нет собак, то после смерти я хочу отправиться туда, куда ушли они.
Predicted: если я не в школьной , , , когда захочешь я туда , куда они пошли . <eos>
--------------------------------------------------
Source: i'll wash the dog.
Target: я искупаю собаку.
Predicted: я помою собаку . <eos>
--------------------------------------------------
Source: do you have a car?
Target: у вас машина есть?
Predicted: у вас есть машина есть машине ? <eos>
--------------------------------------------------
Source: i'm glad you're here today.
Target: я рада, что вы сегодня здесь.
Predicted: рад рад , что ты здесь сегодня . <eos>
--------------------------------------------------


##### Обучим модель с MLPAttention

In [27]:
# Инициализируем механизм внимания
attn = MLPAttention(ENC_HID_DIM, DEC_HID_DIM)

# Инициализируем энкодер и декодер с новым механизмом внимания
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, attn)

# Создаем новую модель Seq2Seq
model_mlp = Seq2Seq(enc, dec, device).to(device)


optimizer_mlp = optim.Adam(model_mlp.parameters())

In [28]:
N_EPOCHS = 10
CLIP = 1

for epoch in range(N_EPOCHS):
    train_loss = train(model_mlp, train_loader, optimizer_mlp, criterion, CLIP)
    val_loss = evaluate(model_mlp, val_loader, criterion)

    print(f'Эпоха: {epoch+1:02}')
    print(f'\tПотеря на обучении: {train_loss:.3f}')
    print(f'\tПотеря на валидации: {val_loss:.3f}')

Эпоха: 01
	Потеря на обучении: 2.748
	Потеря на валидации: 2.878
Эпоха: 02
	Потеря на обучении: 2.299
	Потеря на валидации: 2.850
Эпоха: 03
	Потеря на обучении: 2.253
	Потеря на валидации: 2.957
Эпоха: 04
	Потеря на обучении: 2.263
	Потеря на валидации: 3.101
Эпоха: 05
	Потеря на обучении: 2.289
	Потеря на валидации: 3.063
Эпоха: 06
	Потеря на обучении: 2.330
	Потеря на валидации: 3.229
Эпоха: 07
	Потеря на обучении: 2.416
	Потеря на валидации: 3.242
Эпоха: 08
	Потеря на обучении: 2.492
	Потеря на валидации: 3.452
Эпоха: 09
	Потеря на обучении: 2.600
	Потеря на валидации: 3.469
Эпоха: 10
	Потеря на обучении: 2.702
	Потеря на валидации: 3.498


In [29]:
# Вычисляем BLEU Score для модели с MLPAttention
bleu_score_mlp = calculate_bleu(model_mlp, val_dataset, tokenize_eng, tokenize_rus, eng_vocab, rus_vocab)
print(f'BLEU Score with MLP Attention = {bleu_score_mlp*100:.2f}')

BLEU Score with MLP Attention = 16.45


In [30]:
for i in range(5):
    src_sentence = val_dataset.src_sentences[i]
    trg_sentence = val_dataset.trg_sentences[i]
    
    pred_trg = translate_sentence(model_mlp, src_sentence, tokenize_eng, eng_vocab, rus_vocab)
    pred_sentence = ' '.join(pred_trg)
    
    print(f"Source: {src_sentence}")
    print(f"Target: {trg_sentence}")
    print(f"Predicted with MLP Attention: {pred_sentence}")
    print("-" * 50)

Source: it seems you were right.
Target: похоже, ты была права.
Predicted with MLP Attention: похоже , похоже прав . <eos>
--------------------------------------------------
Source: if there are no dogs in heaven, then when i die i want to go where they went.
Target: если в раю нет собак, то после смерти я хочу отправиться туда, куда ушли они.
Predicted with MLP Attention: если бы нет в от гостей , куда я хочу . <eos>
--------------------------------------------------
Source: i'll wash the dog.
Target: я искупаю собаку.
Predicted with MLP Attention: я помою собаку собаку . <eos>
--------------------------------------------------
Source: do you have a car?
Target: у вас машина есть?
Predicted with MLP Attention: у тебя есть машина машины <eos>
--------------------------------------------------
Source: i'm glad you're here today.
Target: я рада, что вы сегодня здесь.
Predicted with MLP Attention: я рад , что сегодня здесь сегодня . <eos>
--------------------------------------------------

In [31]:
# Вывод BLEU Score для обеих моделей
print(f'BLEU Score with Dot-product Attention = {bleu_score*100:.2f}')
print(f'BLEU Score with MLP Attention = {bleu_score_mlp*100:.2f}')

BLEU Score with Dot-product Attention = 19.87
BLEU Score with MLP Attention = 16.45


### Вывод

В этой работе я выполнил задачу перевода с английского на русский с использованием двух механизмов внимания: на основе скалярного произведения (DotProductAttention) и на основе многоуровневого перцептрона (MLPAttention). Обе модели были обучены на англо-русских парах предложений, а качество их работы я оценил с помощью BLEU Score и примеров переводов.

**Результаты:**
1) BLEU Score:

    - Модель с DotProductAttention показала BLEU Score = 19.87.
    - Модель с MLPAttention показала более низкий результат BLEU Score = 16.45.
      
2) Примеры переводов:

    - Переводы, выполненные моделью с DotProductAttention, были более осмысленными, хотя в некоторых случаях встречались повторения слов и небольшие ошибки.
    - Модель с MLPAttention часто допускала повторение слов и создавала менее логичные конструкции предложений. Например, в переводе вопроса "Do you have a car?" было неправильное повторение слов: "у тебя есть машина машины".

3) Сравнение:

    - Модель с DotProductAttention показала себя более точной и стабильной, как по BLEU Score, так и по качеству переводов. Она генерировала более осмысленные фразы, хотя и не без ошибок.
    - Модель с MLPAttention, несмотря на свою гибкость, продемонстрировала худшие результаты. Это может указывать на то, что MLP требует более тонкой настройки или большего объёма данных для достижения результатов, сравнимых с DotProductAttention.

В ходе работы я осознал, что для повышения качества перевода необходимо:  
    - **Настроить гиперпараметры модели:** Возможно, стоит изменить скорость обучения, размер батча или использовать методы регуляризации, чтобы избежать переобучения.  
    - **Увеличить количество эпох обучения:** Но при этом необходимо контролировать переобучение и, при необходимости, использовать раннюю остановку.  
    - **Использовать предобученные эмбеддинги:** Внедрение GloVe, FastText или других предобученных эмбеддингов может улучшить качество представлений слов и, соответственно, перевода.  
    - **Применить подсловные токены:** Использование методов, таких как Byte Pair Encoding (BPE), может помочь модели лучше обрабатывать редкие или незнакомые слова.  
    
В целом, проделанная работа позволила мне глубже понять принципы работы моделей Seq2Seq с механизмом внимания и особенности их обучения на реальных данных. Полученные результаты служат основой для дальнейших экспериментов и улучшений модели. Я планирую продолжить исследования в этом направлении, внедрить предложенные улучшения и оценить их влияние на качество перевода.